In [1]:
%load_ext autoreload
%autoreload 2
import torch
from cosypose.config import LOCAL_DATA_DIR
from cosypose.datasets.datasets_cfg import make_scene_dataset
from cosypose.rendering.bullet_scene_renderer import BulletSceneRenderer
from cosypose.visualization.singleview import make_singleview_prediction_plots, filter_predictions
from cosypose.visualization.singleview import filter_predictions
from bokeh.plotting import gridplot
from bokeh.io import show, output_notebook; output_notebook()
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Setting OMP and MKL num threads to 1.
pybullet build time: Sep  9 2021 15:08:49


Loading BokehJS ...

In [3]:
# result_id = 'tless-siso-n_views=1--684390594'
# ds_name, urdf_ds_name = 'tless.primesense.test', 'tless.cad'
# pred_key = 'pix2pose_detections/refiner/iteration=4'

result_id = 'tless-siso-n_views=1--4312481950'
ds_name, urdf_ds_name = 'tless.primesense.test', 'tless.cad'
pred_key = 'pix2pose_detections/refiner/iteration=4'

# result_id = 'ycbv-n_views=1--5154971130'
# ds_name, urdf_ds_name = 'ycbv.test.keyframes', 'ycbv'
# pred_key = 'posecnn_init/refiner/iteration=2'

results = LOCAL_DATA_DIR / 'results' / result_id / 'results.pth.tar'
scene_ds = make_scene_dataset(ds_name)
results = torch.load(results)['predictions']

0:00:14.009019 - Building index and loading annotations...


[Memory]14.4s, 0.2min   : Loading build_index...


In [4]:
results[pred_key].infos.loc[:, ['scene_id', 'view_id']].groupby('scene_id').first()

,view_id
scene_id,
1,439
2,307
3,353
4,27
5,330
6,170
7,412
8,300
9,190


In [7]:
scene_id, view_id = 15, 270

# Replace this here, you can use the dataframe above to get examples of scene/view ids.
# scene_id, view_id = 48, 733

this_preds = filter_predictions(results[pred_key], scene_id, view_id)
renderer = BulletSceneRenderer(urdf_ds_name)
figures = make_singleview_prediction_plots(scene_ds, renderer, this_preds)
renderer.disconnect()
# print(this_preds)

show(figures['input_im'])
show(figures['pred_overlay'])

EGL device choice: 0 of 4 (from EGL_VISIBLE_DEVICES)


Loaded EGL 1.5 after reload.
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=GeForce RTX 2060/PCIe/SSE2
GL_VERSION=4.6.0 NVIDIA 460.91.03
GL_SHADING_LANGUAGE_VERSION=4.60 NVIDIA
Version = 4.6.0 NVIDIA 460.91.03
Vendor = NVIDIA Corporation
Renderer = GeForce RTX 2060/PCIe/SSE2
ven = NVIDIA Corporation
PandasTensorCollection(
    poses: torch.Size([6, 4, 4]) torch.float32 cpu,
    poses_input: torch.Size([6, 4, 4]) torch.float32 cpu,
    K_crop: torch.Size([6, 3, 3]) torch.float32 cpu,
    boxes_rend: torch.Size([6, 4]) torch.float32 cpu,
    boxes_crop: torch.Size([6, 4]) torch.float32 cpu,
    distortions: torch.Size([6, 4]) torch.float32 cpu,
    coarse_predictions: torch.Size([6, 4, 4]) torch.float32 cpu,
    initial_bboxes: torch.Size([6, 4]) torch.float32 cpu,
----------------------------------------
    infos:
   scene_id  view_id     score       label  det_id  batch_im_id  group_id  \
0        15      270  1.000000  obj_000028   34859            0      7218   
1        15      270  0.92